# 制作你自己的 **AI** 动作识别系统

设备：香橙派 Zero 3 （同时执行太多任务可能会导致设备卡顿）

## GNU/Linux 的基本操作

>GNU 是 "GNU's Not Unix!" 的递归缩写

`ctrl + alt + t` 打开终端

`ctrl + alt + F1-F7` 切换虚拟终端 tty1-7, 通常tty7号是图形界面

`ctrl + c` 结束指令，终止运行

`ls` 查看此目录下的所有文件与文件夹，尝试 `ls -a`, `ls -l`, `ls -lh` （-h = --human-readable，人类可读）

`man` *manual* 或指令后面加 `--help` 查看指令说明书 

`pwd` *print working directory* 输出所在目录

`code .` (装好vscode的前提下) 从此目录打开vscode






## 配置开发环境

In [11]:
import os
# Get the current working directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")
# Change the directory 切换执行路径
new_dir = "./inference_prediction"
os.chdir(new_dir)
# Verify the new directory 确认执行路径
changed_dir = os.getcwd()
print(f"Changed directory: {changed_dir}")

Changed directory: /Users/promcrdog/Documents/Vector lab/Class - pose detection/Pi_Pose_Prediction/inference_prediction


In [ ]:
!sh setup.sh

## 初步测试

### 找到摄像头编号index

In [ ]:
import cv2

for camera_id in range(0, 20):
    cap = cv2.VideoCapture(camera_id)
    if cap is None or not cap.isOpened():
        # print(f"No camera found at index {camera_id}")
        continue
    else:
        ret, frame = cap.read()
        if ret:
            print(f"camera found at index {camera_id}")
        cap.release()

cv2.destroyWindow

### 测试

In [3]:
# Hardcoded variables 硬编变量
ESTIMATION_MODEL = 'movenet_lightning'  # 估计模型
TRACKER_TYPE = 'bounding_box'  # 跟踪器类型 ('keypoint' or 'bounding_box')
CLASSIFICATION_MODEL = 'posenet.tflite'  # Set to None if not using a classifier 如果不使用分类器，设置为None
LABEL_FILE = 'labels.txt'  # 标签文件
CAMERA_ID = 0  # 相机ID
WIDTH = 640  # 宽度
HEIGHT = 480  # 高度

In [ ]:
import logging
import sys
import time


import cv2
from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet
import utils as utils

# Change the directory
new_dir = "./inference_prediction"
os.chdir(new_dir)

def run(estimation_model: str, tracker_type: str, classification_model: str,
        label_file: str, camera_id: int, width: int, height: int) -> None:
    """
    Continuously run inference on images acquired from the camera.
    连续对从相机获取的图像进行推理。
    """

    # Notify users that tracker is only enabled for MoveNet MultiPose model.
    # 通知用户跟踪器仅适用于MoveNet MultiPose模型。
    if tracker_type and (estimation_model != 'movenet_multipose'):
        logging.warning(
            'No tracker will be used as tracker can only be enabled for '
            'MoveNet MultiPose model.'
            '跟踪器只能用于MoveNet MultiPose模型。'
        )

    # Initialize the pose estimator selected.
    # 初始化选姿势预测器。
    if estimation_model in ['movenet_lightning', 'movenet_thunder']:
        pose_detector = Movenet(estimation_model)
    elif estimation_model == 'posenet':
        pose_detector = Posenet(estimation_model)
    elif estimation_model == 'movenet_multipose':
        pose_detector = MoveNetMultiPose(estimation_model, tracker_type)
    else:
        sys.exit('ERROR: Model is not supported. 错误：不支持该模型。')

    # Variables to calculate FPS
    # 计算FPS的变量
    counter, fps = 0, 0
    start_time = time.time()

    # Start capturing video input from the camera
    # 开始从相机捕获视频输入
    cap = cv2.VideoCapture(camera_id)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

    # Visualization parameters
    # 可视化参数
    row_size = 20  # pixels 像素
    left_margin = 24  # pixels 像素
    text_color = (0, 0, 255)  # red 红色
    font_size = 1
    font_thickness = 1
    classification_results_to_show = 3
    fps_avg_frame_count = 10
    keypoint_detection_threshold_for_classifier = 0.1
    classifier = None

    # Initialize the classification model
    # 初始化分类模型
    if classification_model:
        classifier = Classifier(classification_model, label_file)
        classification_results_to_show = min(classification_results_to_show,
                                             len(classifier.pose_class_names))

    # Continuously capture images from the camera and run inference
    # 持续从相机捕获图像并进行推理
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            sys.exit(
                'ERROR: Unable to read from webcam. Please verify your webcam settings.'
                '错误：无法从网络摄像头读取。请检查您的网络摄像头设置。'
            )

        counter += 1
        image = cv2.flip(image, 1)

        if estimation_model == 'movenet_multipose':
            # Run pose estimation using a MultiPose model.
            # 使用MultiPose模型进行姿势估计。
            list_persons = pose_detector.detect(image)
        else:
            # Run pose estimation using a SinglePose model, and wrap the result in an array.
            # 使用SinglePose模型进行姿势估计，并将结果包装在数组中。
            list_persons = [pose_detector.detect(image)]

        # Draw keypoints and edges on input image
        # 在输入图像上绘制关键点和边缘
        image = utils.visualize(image, list_persons)

        if classifier:
            # Check if all keypoints are detected before running the classifier.
            # If there's a keypoint below the threshold, show an error.
            # 在运行分类器之前检查是否检测到所有关键点。
            # 如果有关键点低于阈值，显示错误。
            person = list_persons[0]
            min_score = min([keypoint.score for keypoint in person.keypoints])
            if min_score < keypoint_detection_threshold_for_classifier:
                error_text = 'Some keypoints are not detected. 一些关键点未被检测到。'
                text_location = (left_margin, 2 * row_size)
                cv2.putText(image, error_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                            font_size, text_color, font_thickness)
                error_text = 'Make sure the person is fully visible in the camera. 确保相机中完全可见人物。'
                text_location = (left_margin, 3 * row_size)
                cv2.putText(image, error_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                            font_size, text_color, font_thickness)
            else:
                # Run pose classification
                # 运行姿势分类
                prob_list = classifier.classify_pose(person)

                # Show classification results on the image
                # 在图像上显示分类结果
                for i in range(classification_results_to_show):
                    class_name = prob_list[i].label
                    probability = round(prob_list[i].score, 2)
                    result_text = class_name + ' (' + str(probability) + ')'
                    text_location = (left_margin, (i + 2) * row_size)
                    cv2.putText(image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                                font_size, text_color, font_thickness)

        # Calculate the FPS
        # 计算FPS
        if counter % fps_avg_frame_count == 0:
            end_time = time.time()
            fps = fps_avg_frame_count / (end_time - start_time)
            start_time = time.time()

        # Show the FPS
        # 显示FPS
        fps_text = 'FPS = ' + str(int(fps))
        text_location = (left_margin, row_size)
        cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    font_size, text_color, font_thickness)

        # Stop the program if the ESC key is pressed.
        # 如果按下ESC键，停止程序。
        if cv2.waitKey(1) == 27:
            break
        cv2.imshow(estimation_model, image)

    cap.release()
    cv2.destroyAllWindows()

def main():
    """
    Main function to run the pose estimation and classification.
    运行姿势估计和分类的主函数。
    """
    run(ESTIMATION_MODEL, TRACKER_TYPE, CLASSIFICATION_MODEL, LABEL_FILE, CAMERA_ID, WIDTH, HEIGHT)

if __name__ == '__main__':
    main()

In [ ]:
# Change back to initiall working directory
os.chdir(current_dir)